## Naive Bayes: Limes and Lemons Shop

![Alt Text](img.png)

In [2]:
import sqlite3
import pandas as pd

### Load the Data

The input data consists of the following tables:

1. `transactions`: Contains the transaction_id, product_id, and quantity of products purchased.
2. `product_stock_outs`: Contains the product_id and the date_time when the product was out of stock.
3. `substitution_groups`: Contains the substitution_group_id and the product_ids that are substitutes of each other.
4. `transaction_spine`: Contains a list of all possible combinations of transactions and items within the substitution group. This is mainly used as the spine to find the availability of every product during a transaction.
5. `transaction_outcome`: Contains the outcome of each transaction, i.e., which product was purchased
6. `transaction_availability`: Contains the transaction_id, product_id_available, and the substitution_group_id.

The Naive Bayes model consists requires the following tables:

7. `priors`: Contains the prior probabilities of purchasing a product.
8. `likelihoods`: Contains the likelihoods of observing the evidence given the hypothesis.
9. `posteriors`: Contains the posterior probabilities of purchasing a product given the evidence.

The output data consists of:

10. `transactions_corrected`: Contains the corrected transactions after applying the substitution correction.

In [27]:
def execute_sql_file(filepath, connection):
    with open(filepath, 'r') as file:
        sql_script = file.read()
    connection.executescript(sql_script)
    print(f"Executed {filepath}")

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

In [28]:
# Load input data
execute_sql_file('../data/input/1_transactions.sql', conn)
execute_sql_file('../data/input/2_product_stock_outs.sql', conn)
execute_sql_file('../data/input/3_substitution_groups.sql', conn)
execute_sql_file('../data/input/4_transaction_spine.sql', conn)
execute_sql_file('../data/input/5_transaction_outcome.sql', conn)
execute_sql_file('../data/input/6_transaction_availability.sql', conn)

# Load naive bayes model
execute_sql_file('../data/naive_bayes/7_priors.sql', conn)
execute_sql_file('../data/naive_bayes/8_likelihoods.sql', conn)
execute_sql_file('../data/naive_bayes/9_posteriors.sql', conn)

# Load output data
execute_sql_file('../data/output/10_transactions_corrected.sql', conn)

Executed ../data/input/1_transactions.sql
Executed ../data/input/2_product_stock_outs.sql
Executed ../data/input/3_substitution_groups.sql
Executed ../data/input/4_transaction_spine.sql
Executed ../data/input/5_transaction_outcome.sql
Executed ../data/input/6_transaction_availability.sql
Executed ../data/naive_bayes/7_priors.sql
Executed ../data/naive_bayes/8_likelihoods.sql
Executed ../data/naive_bayes/9_posteriors.sql
Executed ../data/output/10_transactions_corrected.sql


### Explore the Data

Let's explore the transaction data to determine whether there is a potential substitution effect between limes and lemons.

At first glance, the data suggests a strong customer preference for lemons over limes.

In [16]:
df_transactions = pd.read_sql_query('SELECT * FROM transactions', conn)
df_transactions.head()

,transaction_id,sales_date_time,product_name,product_id,quantity_sold
0,1,2024-10-01 13:15:00,lime,11,2
1,2,2024-10-01 13:20:00,lemon,12,1
2,3,2024-10-01 14:50:00,lime,11,3
3,4,2024-10-01 14:55:00,lemon,12,1
4,5,2024-10-01 15:00:00,lemon,12,3


In [17]:
# look at nr of transactions per product
df_transactions.groupby('product_name').size().reset_index(name='nr_transactions')

,product_name,nr_transactions
0,lemon,23
1,lime,14


In [18]:
# Find transactions made when there were stock outs
df_out_of_stock = pd.read_sql_query("""
    SELECT DISTINCT 
        transaction_id,  
        CASE WHEN product_id_outcome = 11 THEN 'lime' ELSE 'lemon' END AS product_purchased,
        CASE WHEN product_id_available = 11 THEN 'lime' ELSE 'lemon' END AS product_stock_out,
        is_available
    FROM transaction_outcome
    INNER JOIN transaction_availability USING (transaction_id)
    WHERE is_available = 0
""", conn)

df_out_of_stock


,transaction_id,product_purchased,product_stock_out,is_available
0,19,lemon,lime,0
1,20,lemon,lime,0
2,21,lemon,lime,0
3,22,lemon,lime,0
4,23,lemon,lime,0
5,24,lemon,lime,0
6,25,lemon,lime,0
7,26,lemon,lime,0


### Define the Problem

In our dataset, we observed **8 transactions** where customers purchased lemons while limes were out of stock. Because lemons and limes are likely substitutes, we want to estimate the probability that a customer will purchase lemons specifically because limes are unavailable.

To do this, we apply Naive Bayes to compute the posterior probability of purchasing lemons, given that limes are out of stock. The formula is:

$$ \mathbb{P}(\text{Lemons} \mid \text{Limes Out of Stock}) = \frac{\mathbb{P}(\text{Limes Out of Stock} \mid \text{Lemons}) \cdot \mathbb{P}(\text{Lemons})}{\mathbb{P}(\text{Limes  Out of Stock})} $$

Where: 

- $\mathbb{P}(\text{Lemons} \mid \text{Limes Out of Stock})$ is the posterior probability of purchasing lemons given that limes are out of stock.
- $\mathbb{P}(\text{Limes Out of Stock} \mid \text{Lemons})$ is the likelihood of observing that limes are out of stock given that lemons were purchased.
- $\mathbb{P}(\text{Lemons})$ is the prior probability of purchasing lemons.
- $\mathbb{P}(\text{Limes Out of Stock})$ is the normalization constant.

We begin by breaking down the problem and calculating the **priors** and **likelihoods**.




In [19]:
df_priors = pd.read_sql_query("""
SELECT 
    CASE WHEN product_id_outcome = 11 THEN 'lime' ELSE 'lemon' END AS product_name,
    count as n_times_purchased,
    total as n_total_purchased,
    prior
FROM priors
""", conn)

df_priors.head()

,product_name,n_times_purchased,n_total_purchased,prior
0,lime,14,37,0.378378
1,lemon,23,37,0.621622


We calculate this prior as the ratio of the number of transactions involving limes to the total number of transactions observed. For example, if we observed **14 transactions** where limes were purchased out of a total of **37 transactions**, the prior probability of purchasing limes is:

$$ \mathbb{P}(\text{Limes}) = \frac{14}{37} = 0.378378 $$

This means that, on average, there is a **37.8% chance** that a customer will purchase limes, not considering the availability of lemons. With the prior probability of purchasing limes established, we can move on to calculating the likelihoods needed for our Naive Bayes model.

In [20]:
df_likelihoods = pd.read_sql_query("""
WITH outcome_availability AS ( 
    SELECT DISTINCT 
        substitution_group_id,
        transaction_id,
        product_id_outcome,
        product_id_available,
        is_available
    FROM transaction_outcome
    INNER JOIN transaction_availability USING (transaction_id, substitution_group_id)
)

SELECT
    CASE WHEN product_id_outcome = 11 THEN 'lime' ELSE 'lemon' END AS product_purchased,
    CASE WHEN product_id_available = 11 THEN 'lime' ELSE 'lemon' END AS product_stock_out,
    is_available,
    COUNT(*) as n_occurrences_of_stock_out_status,
    SUM(COUNT(*)) OVER (PARTITION BY product_id_outcome, product_id_available) as total,
    CAST(COUNT(*) AS REAL) / CAST(SUM(COUNT(*)) OVER (PARTITION BY substitution_group_id, product_id_outcome, product_id_available) AS REAL) as likelihood
FROM outcome_availability
WHERE product_id_outcome = 12 and product_id_available = 11
GROUP BY 
    product_id_outcome,
    product_id_available,
    is_available
""", conn)

df_likelihoods

,product_purchased,product_stock_out,is_available,n_occurrences_of_stock_out_status,total,likelihood
0,lemon,lime,0,8,23,0.347826
1,lemon,lime,1,15,23,0.652174


We are interested here in the likelihood of **limes being out of stock** given that a customer has purchased **lemons**. This helps us understand how often stock-outs of limes coincide with purchases of lemons.

To compute this likelihood, we need to count two things:

- The number of transactions where limes were out of stock and lemons were purchased (8 occurrences).
- The total number of transactions where lemons were purchased (23 occurrences).

Using this information, the likelihood is calculated as:

$$ \mathbb{P}(\text{Limes Out  of Stock} \mid \text{Lemons}) = \frac{8}{23} = 0.3478 $$

This value tells us that in about 34.8% of lemon purchases, limes were out of stock.

With both the prior and likelihood now calculated, we have all the components to compute the posterior probability.

### Calculate the Posterior Probability

The posterior probability is the probability that a customer purchases lemons, given that limes are out of stock. As we mentioned before, this is calculated using the formula:

$$ \mathbb{P}(\text{Lemons} \mid \text{Limes Out  of Stock}) = \frac{\mathbb{P}(\text{Limes  Out  of  Stock} \mid \text{Lemons}) \cdot \mathbb{P}(\text{Lemons})}{\mathbb{P}(\text{Limes  Out  of  Stock})} $$

Before we can calculate the posterior, we need to determine one remaining term: **the normalization constant** $\mathbb{P}(\text{Limes  Out  of  Stock})$. This ensures that the posterior probabilities sum to 1. We calculate it as the sum of the likelihoods, weighted by the priors for each product:

$$ \mathbb{P}(\text{Limes  Out  of  Stock}) = \mathbb{P}(\text{Lemons}) \cdot \mathbb{P}(\text{Limes  Out  of  Stock} \mid \text{Lemons}) + \mathbb{P}(\text{Limes}) \cdot \mathbb{P}(\text{Limes  Out  of  Stock} \mid \text{Limes}) $$

However, since **limes cannot be out of stock when limes are purchased**, the term $ \mathbb{P}(\text{Limes}) \cdot \mathbb{P}(\text{Limes  Out  of  Stock} \mid \text{Limes}) $ equals 0. This simplifies the normalization constant to:

$$ \mathbb{P}(\text{Limes  Out  of  Stock}) = \mathbb{P}(\text{Lemons}) \cdot \mathbb{P}(\text{Limes  Out  of  Stock} \mid \text{Lemons}) $$

Now, substituting the known values:

$$ \mathbb{P}(\text{Limes  Out  of  Stock}) = 0.6216 \cdot 0.3478 = 0.2162 $$

Finally, we can calculate the posterior probability:

$$ \mathbb{P}(\text{Lemons} \mid \text{Limes  Out  of  Stock}) = \frac{0.3478 \cdot 0.6216}{0.2162} = 1 $$



In [15]:
df_posteriors = pd.read_sql_query("""
SELECT 
    transaction_id,
    posterior,
    posterior_base,
    substitution_correction_ratio
FROM posteriors
WHERE posterior > posterior_base
""", conn)
df_posteriors

,transaction_id,posterior,posterior_base,substitution_correction_ratio
0,19,1.0,0.577,0.577
1,20,1.0,0.577,0.577
2,21,1.0,0.577,0.577
3,22,1.0,0.577,0.577
4,23,1.0,0.577,0.577
5,24,1.0,0.577,0.577
6,25,1.0,0.577,0.577
7,26,1.0,0.577,0.577


### Correcting the sales

The final step in our process is to correct the observed transactions by applying the **substitution correction ratio**. By applying the correction ratio to the original quantity of lemons sold, we can estimate the **true demand**.

From the previous steps, we calculated that the posterior probability of purchasing lemons when limes are out of stock is **1**.

The **substitution correction ratio** is calculated as follows:

$$ \text{substitution correction ratio} = \frac{P(\text{Lemons} \mid \text{Lemons and limes in stock})}{P(\text{Lemons} \mid \text{Limes out of stock})} = \frac{0.577}{1} = 0.577 $$

This ratio reflects that the true demand for lemons is lower when both lemons and limes are in stock. Next, we multiply the observed quantity by this correction ratio.

For example, in **transaction_id 19**, where 3 lemons were purchased, the corrected quantity is:

$$ \text{quantity corrected} = \text{quantity} \cdot \text{substitution correction ratio} = 3 \cdot 0.577 = 1.731 $$

In [21]:
df_transactions_corrected = pd.read_sql_query("""
SELECT 
    transaction_id,
    quantity,
    quantity_without_sosc,
    substitution_correction_ratio
FROM transactions_corrected
WHERE substitution_correction_ratio < 1
""", conn)
df_transactions_corrected

,transaction_id,quantity,quantity_without_sosc,substitution_correction_ratio
0,19,1.731,3,0.577
1,20,2.308,4,0.577
2,21,1.154,2,0.577
3,22,1.154,2,0.577
4,23,1.154,2,0.577
5,24,0.577,1,0.577
6,25,0.577,1,0.577
7,26,1.731,3,0.577


In [31]:
df_demand_lemons = pd.read_sql_query("""
SELECT 
    product_id,
    product_name,
    SUM(quantity) as demand_with_correction,
    SUM(quantity_without_sosc) as demand_without_correction
FROM transactions_corrected
WHERE product_id = 12
GROUP BY product_id
""", conn)
df_demand_lemons

,product_id,product_name,demand_with_correction,demand_without_correction
0,12,lemon,43.386,51
